In [12]:
import rtree
import fiona.crs
import geopandas as gpd

In [13]:
index = rtree.Rtree()
zones = gpd.read_file('NYCHA/NYCHA_developments_July2011.shp').to_crs(fiona.crs.from_epsg(2263))

In [14]:
zones.head()

,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11,geometry
0,VARIOUS,16,FHA REPOSSESSED HOUSES (GROUP III),0,18,213,40,"(POLYGON ((1002120.103683472 179552.644237477,..."
1,VARIOUS,19,FHA REPOSSESSED HOUSES (GROUP IV),0,22,226,60,(POLYGON ((1003910.250693723 180500.6387325663...
2,VARIOUS,40,FHA REPOSSESSED HOUSES (GROUP V),0,50,260,103,"(POLYGON ((1005655.023450882 178293.417425233,..."
3,QUEENS,9,FHA REPOSSESSED HOUSES (GROUP VI),0,14,273,22,(POLYGON ((1046846.522760647 181178.3473508989...
4,QUEENS,11,FHA REPOSSESSED HOUSES (GROUP VII),0,17,274,22,(POLYGON ((1048185.867194804 180368.3889970588...


In [15]:
import pandas as pd
f = pd.read_csv('cvh_people.csv')
df = pd.DataFrame(f)

In [19]:
df['coords'] = list(zip(df.Latitude, df.Longitude))

In [20]:
df.head()

,Internal Contact ID,Latitude,Longitude,coords
0,5,40.687482,-73.963384,"(40.687482, -73.963384)"
1,226,40.769909,-73.992111,"(40.769909, -73.992111)"
2,228,40.746794,-73.982509,"(40.7467938, -73.9825089)"
3,230,40.823687,-73.868344,"(40.8236871, -73.8683435)"
4,232,40.760444,-73.973270,"(40.760444, -73.97327)"


In [21]:
from shapely.geometry import Point

In [29]:
df['geometry'] = df.coords.apply(lambda x: Point(x))

In [37]:
df = df[['Internal Contact ID', 'geometry']]

In [49]:
df.crs = {'init': 'epsg:2263', 'no_defs': True}

In [50]:
df.head()

,Internal Contact ID,geometry
0,5,POINT (40.687482 -73.96338399999999)
1,226,POINT (40.76990900000001 -73.99211099999999)
2,228,POINT (40.7467938 -73.9825089)
3,230,POINT (40.8236871 -73.86834350000001)
4,232,POINT (40.760444 -73.97327)
